In [ ]:
# snakemake preamble inserted here

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from snco import MarkerRecords, PredictionRecords
from snco.records import NestedData

### Report for analysis of {{ snakemake.wildcards["dataset_name"] }}

In [ ]:
markers = MarkerRecords.read_json(snakemake.input.markers)
markers

In [ ]:
haplotypes = PredictionRecords.read_json(snakemake.input.preds)
haplotypes.add_metadata(
    total_marker_count=NestedData(
        ['cb'], int, {cb: int(markers.total_marker_count(cb)) for cb in markers.barcodes}
    )
)
haplotypes

In [ ]:
cb_stats = pd.read_csv(snakemake.input.stats, sep='\t', index_col='cb')
cb_stats.head()

## Number of informative reads

The number of reads that distinguish haplotypes is dependent on the sequencing stragety, library complexity, and sequencing depth, as well as the number of (mappable) SNPs that actually distinguish the two haplotypes. This dataset has already been filtered to remove barcodes with fewer than {{ snakemake.config["haplotyping"]["preprocessing"]["min_informative_reads_per_barcode"] }} informative reads.

In [ ]:
marker_counts = [markers.total_marker_count(cb) for cb in markers.barcodes]
fig, ax = plt.subplots()
ax.hist(
    marker_counts,
    bins=np.logspace(np.log10(min(marker_counts)), np.log10(max(marker_counts)), 25),
)
plt.xscale('log')
ax.set_xlabel('Number of informative reads (log10 scale)')
ax.set_ylabel('Number of barcodes/samples')
ax.set_title(f'{snakemake.wildcards.dataset_name} informative read distribution')
plt.show()

Sparsely sequenced nuclei will have fewer estimated crossovers due to failure to detect some, especially at chromosome ends. Nuclei with large numbers of reads and large crossover estimates likely represent doublets. We can use the scatterplot of crossover rate vs read number to estimate a suitable threshold.

In [ ]:
fig, ax =plt.subplots(figsize=(8, 8))
ax = sns.regplot(
    x=10 ** cb_stats.co_n_marker_reads,
    y=cb_stats.n_crossovers,
    data=cb_stats,
    scatter_kws=dict(alpha=0.05),
    line_kws=dict(color='#252525'),
    lowess=True,
    ax=ax
)
plt.xscale('log')
ax.set_xlabel('Number of informative reads (log10 scale)')
ax.set_ylabel('Estimated Crossovers')
plt.show()

## Doublet probabilities

`snco` performs doublet detection for single cell datasets using "synthetic doublets", created by mixing multiple barcodes together. Several summary statistics are calculated and the K-nearest neighbours classification is used to produce a score (for real data vs synthetic doublets). The choice of filtering threshold to remove doublets can vary with dataset/application and is left up to the user.

In [ ]:
# set a threshold for removing sparsely sequenced nuclei here
non_sparse_haplotypes = haplotypes.query('total_marker_count > 300')
non_sparse_markers = markers.filter(non_sparse_haplotypes.barcodes, inplace=False)


def qcutter(q, doublet_probs):
    x = np.asarray(list(doublet_probs.values()))
    quantiles = np.linspace(0, 100, q + 1)
    bins = np.percentile(x, quantiles)
    bins[0] -= 1e-8  # ensure inclusion of min value
    labels = [f'[{bins[i]:.2f}, {bins[i+1]:.2f}]' for i in range(q)]
    def _wrapped(cb):
        i = np.digitize(doublet_probs[cb], bins[1:-1], right=True)
        return labels[i]
    return _wrapped

if hasattr(non_sparse_haplotypes, 'doublet_probability'):
    plt.hist(
        list(non_sparse_haplotypes.doublet_probability.values()),
        bins=25, range=(0, 1),
    )
    plt.show()
    quantile_separated_barcodes = dict(non_sparse_markers.groupby(qcutter(5, non_sparse_haplotypes.doublet_probability)))
    for i, q in enumerate(sorted(quantile_separated_barcodes)):
        markers_group = quantile_separated_barcodes[q]
        cb = np.random.choice(markers_group.barcodes)
        fig, axes = markers_group.plot_barcode(cb, co_preds=non_sparse_haplotypes, max_yheight=10, figsize=(12, 3))
        fig.suptitle(f'Doublet probability q{i+1} {q} example barcode')
        plt.tight_layout()
        plt.show()

## Filtering

You can create your own quality control filters here - this is just an example that is generally suitable for 10x RNA data

In [ ]:
if hasattr(non_sparse_haplotypes, 'doublet_probability'):
    haplotypes_filt = haplotypes.query('(doublet_probability < 0.25) & (total_marker_count > 300)')
else:
    haplotypes_filt = haplotypes.query('total_marker_count > 300')
markers_filt = markers.filter(haplotypes_filt.barcodes, inplace=False)
cb_stats_filt = cb_stats.loc[haplotypes_filt.barcodes].copy()

### Segregation distortion

We can plot the overall allele frequencies of the dataset to look for single locus distortions (these can sometimes be affected by noise in the data, if you see weird/extreme patterns then perhaps stricter data cleaning/filtering is required).

In [ ]:
haplotypes_filt.plot_allele_ratio()
plt.show()

### Crossover distributions

In [ ]:
fig, ax = plt.subplots()
ax.hist(cb_stats_filt.n_crossovers, bins=20)
ax.set_xlabel('Estimated crossovers')
ax.set_ylabel('Number of barcodes/samples')
ax.set_title(f'{snakemake.wildcards.dataset_name} estimated crossover numbers')
plt.show()

haplotypes_filt.plot_recombination_landscape(co_markers=markers_filt)
plt.show()